In [1]:
from tinytensor.engine import Tensor
from tinytensor.nn import Model, Dense, mse, l2_reg, array_to_Tensor_list, sgd
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs


In [ ]:
# Make dataset
X, y = make_moons(n_samples=100, noise=0.1)
y = y.reshape(-1,1)

print("X shape:",X.shape)
print("y shape:",y.shape)

# Create a figure with subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 4))  # 1 row, 2 columns

# Scatter plot on the left
axs[0].scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')
axs[0].set_title('Scatter Plot')

# Histogram on the right
axs[1].hist(y, bins=20, color='blue', alpha=0.7)
axs[1].set_title('Histogram')

# Display the figure
plt.show()


In [ ]:
# setup the model
Xt = array_to_Tensor_list(X)
yt = array_to_Tensor_list(y)

initial_model = Model()
initial_model.layers.append(Dense(2,16,"relu"))
initial_model.layers.append(Dense(16,16,"relu"))
initial_model.layers.append(Dense(16,1,"sigmoid"))


In [ ]:
def loss(model: Model,x: list,y: list) -> Tuple[Tensor,float]:
        
    #forward pass
    yhats = list(map(model,x))

    #loss functions
    data_loss = mse(yhats,y)
    total_loss = data_loss

    #calculate accuracy
    accuracies = [(yhat.data > 0.5) == (yi.data > 0.5) for yhat,yi in zip(yhats,y)]
    accuracy = sum(accuracies) / len(accuracies)

    return total_loss,accuracy

def train(model: Model,X: list,y: list) -> list:
    
    total_loss,accuracy = loss(model,X,y)
    print(f"total_loss: {total_loss.data}; accuracy:{accuracy}")

    #backward
    total_loss.backward()

    #TODO: return grads here
    #get new layers
    grads = [layer.grads() for layer in model.layers]
    new_layers = [sgd(layer,grad,10) for layer,grad in zip(model.layers,grads)]
    new_model = Model(layers = new_layers)
    
    return new_model

In [ ]:
def train_loop(model: Model,X: list,y: list,num_iterations: int) -> Model:
    for k in range(num_iterations):
        new_model = train(model,X,y)
        model = new_model

    return model

final_model = train_loop(initial_model,Xt,yt,num_iterations = 100)

In [ ]:
Xt[0].data